我要带着你用 Sequential Chain 把几个 LLMChain 串起来，形成一个顺序链。

这个示例中，我们的目标是这样的：
- 第一步，我们假设大模型是一个植物学家，让他给出某种特定鲜花的知识和介绍。
- 第二步，我们假设大模型是一个鲜花评论者，让他参考上面植物学家的文字输出，对鲜花进行评论。
- 第三步，我们假设大模型是易速鲜花的社交媒体运营经理，让他参考上面植物学家和鲜花评论者的文字输出，来写一篇鲜花运营文案。

In [6]:
# 导入所需要的库
from langchain_openai import AzureChatOpenAI
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
import os

In [7]:
# 第一个LLMChain：生成鲜花的介绍

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("BASE_URL"),
    openai_api_version=os.getenv("API_VERSION"),
    deployment_name=os.getenv("DEPLOYMENT_NAME"),
    openai_api_key=os.getenv("API_KEY"),
    temperature=0.7
)

template = """
你是一个植物学家。给定花的名称和类型，你需要为这种花写一个200字左右的介绍。
花名: {name}
颜色: {color}
植物学家: 这是关于上述花的介绍:"""

prompt_template = PromptTemplate(
    input_variables=["name", "color"],
    template=template
)

introduction_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_key="introduction"
)

c:\Users\I078212\workspace\eagle\langchain-in-action\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [8]:
# 第二个LLMChain：根据鲜花的介绍写出鲜花的评论
template = """
你是一位鲜花评论家。给定一种花的介绍，你需要为这种花写一篇200字左右的评论。
鲜花介绍:
{introduction}
花评人对上述花的评论:"""
prompt_template = PromptTemplate(
    input_variables=["introduction"],
    template=template
)
review_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_key="review"
)

In [9]:
# 第三个LLMChain：根据鲜花的介绍和评论写出一篇自媒体的文案
template = """
你是一家花店的社交媒体经理。给定一种花的介绍和评论，你需要为这种花写一篇社交媒体的帖子，300字左右。
鲜花介绍:
{introduction}
花评人对上述花的评论:
{review}
社交媒体帖子:
"""
prompt_template = PromptTemplate(
    input_variables=["introduction", "review"],
    template=template
)
social_post_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_key="social_post_text"
)

In [10]:
# 总的链：按顺序运行三个链
overall_chain = SequentialChain(
    chains=[introduction_chain, review_chain, social_post_chain],
    input_variables=["name", "color"],
    output_variables=["introduction", "review", "social_post_text"],
    verbose=True
)


In [12]:
# 运行链并打印结果
result = overall_chain.invoke({
    "name": "玫瑰",
    "color": "黑色"
})
print(result)



> Entering new SequentialChain chain...

> Finished chain.
{'name': '玫瑰', 'color': '黑色', 'introduction': '作为植物学家，我要介绍的这种玫瑰花因其独特的颜色而格外引人注目。黑色玫瑰，虽然在自然界中并不存在纯粹的黑色，但却有些品种的玫瑰花拥有如此深的红色或紫色，以至于在特定的光线下看起来几乎是黑色。这种色彩的深邃给予了黑色玫瑰一种神秘和优雅的气质。\n\n玫瑰，学名为Rosa，属于蔷薇科，是全球广泛种植与欣赏的花卉之一。历史上，玫瑰被赋予了许多象征意义，从爱情到友谊，从战争到和平。而黑色玫瑰常常被关联于悼念或告别，使其成为一种在特定场合下表达深沉情感的选择。\n\n在栽培上，黑色玫瑰的培养是一个挑战。光照、土壤pH值、温度和湿度的微妙平衡都会影响花瓣的颜色深浅。此外，这种玫瑰的护理需要细心与专业知识，例如及时修剪、合适的施肥和病虫害的防治。\n\n总之，黑色玫瑰不仅因其罕见的颜色而珍贵，还因其在文化和情感上的深远影响而倍受珍视。对于花卉爱好者和专业园艺师来说，它提供了一种独特的美学体验和栽培上的挑战。', 'review': '在众多玫瑰的色彩斑斓中，黑色玫瑰以其神秘的魅力和深沉的色调，犹如夜幕中的一抹优雅，让人不禁为之着迷。尽管称之为“黑色”并非完全贴切，这些接近黑色的深红或深紫玫瑰却无疑是大自然中的艺术杰作。\n\n这种玫瑰的珍贵不仅在于其独一无二的外观，更在于其所承载的丰富象征意义。在特殊的文化语境中，它代表着告别与纪念，是情感表达的深邃载体。然而，这并不影响它在其他场合传递不同情感的能力，如同玫瑰的其他色彩一样，黑色玫瑰同样能够代表着爱情的深切与力量。\n\n在栽培方面，黑色玫瑰的确是一项考验，它需要园艺师的精心养护和对生长环境的精细调控。它不仅仅是一朵花，更是一位需要细心呵护的高贵客人。对于那些勇于接受挑战的花卉爱好者来说，黑色玫瑰提供了一次难得的学习与成长的机会。\n\n总体而言，黑色玫瑰是大自然中的一个奇迹，是美丽与神秘的完美结合。无论是对于观赏者还是园艺专家，它都是一种不可多得的珍品，值得我们去欣赏、理解和珍惜。', 'social_post_text': '探索花卉世界中的神秘力量，我们今天要介绍的不仅仅是一朵玫瑰，而是一段情感的历程——迷人的